In [1]:
from IPython.display import Image
Image(url='https://vesg.ipsl.upmc.fr/thredds/fileServer/IPSLFS/jservon/CliMAF_Notebooks_html/CliMAF-logo-small.png') 

### A science-oriented framework to ease the analysis of climate model simulations
##### WP5 ANR Convergence
##### Development team: Stéphane Sénési (CNRM-GAME), Gaëlle Rigoudy (CNRM-GAME), Jérôme Servonnat (LSCE-IPSL), Ludivine Vignon (CNRM-GAME), Laurent Franchisteguy (CNRM-GAME), Patrick Brockmann (LSCE-IPSL)
##### Beta-testing: Olivier Marti (LSCE-IPSL), Marie-Pierre Moine (CERFACS), Emilia Sanchez-Gomez (CERFACS)
##   
##### contact: climaf@meteo.fr
##### users list: climaf-users@meteo.fr

# CMIP6 with CliMAF: basis and tips to work on ensembles
In this notebook we will see:
- 1. the basic access to CMIP6 datasets
- 2. how to make different kinds of CMIP6 ensemble:
     - 2.1 all realizations of one model
     - 2.2 all models and one specified realization: dealing with the multiple grids
     - 2.3 all models and the first realization available
     - 2.4 all models and all realizations
- 3. IPSL-CM6_historical-EXT project: working with the historical-EXT IPSL Large Ensemble of historical runs extended with ssp585 until 2059
#  

## First, import climaf

In [2]:
from climaf.api import *

CliMAF version = 1.2.13


CliMAF install => /ciclad-home/jservon/Evaluation/CliMAF/climaf_installs/climaf_V1.2.13_post
python => /modfs/modtools-phw/miniconda2/envs/analyse_2.7/bin/python
---
Required softwares to run CliMAF => you are using the following versions/installations:
ncl 6.6.2 => /modfs/modtools-phw/miniconda2/envs/analyse_2.7/bin/ncl
cdo 1.9.6 => /opt/nco/1.9/bin/cdo
nco (ncks) 4.5.2 => /opt/nco-4.5.2/bin/ncks
ncdump fichier => /modfs/modtools-phw/miniconda2/envs/analyse_2.7/bin/ncdump
Check stamping requirements
nco (ncatted) found -> /opt/nco-4.5.2/bin/ncatted
convert found -> /usr/bin/convert
pdftk found -> /usr/bin/pdftk
exiv2 found -> /ciclad-home/jservon/Evaluation/CliMAF/climaf_installs/climaf_V1.2.13_post/bin/exiv2
---


Cache directory set to : /data/jservon/climafcache (use $CLIMAF_CACHE if set) 
Cache directory for remote data set to : /data/jservon/climafcache/remote_data (use $CLIMAF_REMOTE_CACHE if set) 
Available macros read from ~/.climaf.macros are : []


## And set verbosity ('critical' -> minimum ; 'debug' -> maximum)

In [3]:
clog('critical') # min verbosity = critical < warning < info < debug = max verbosity

### ... and dont' forget to open the documentation in case you have questions.

### http://climaf.readthedocs.org/

### -> Use the "Quick search" space to search for what you are interested in, it is really powerfull!
#  
#  

## 1. Basic access to a CMIP6 dataset and difference with CMIP5

#### New features compared with CMIP5:
- realization (which is actually 'variant') now has the syntax r1i1p1f1 (and not r1i1p1); the f states for 'forcing'
- multiple grids might be available for one variable/model/experiment/table
- the frequency is now specified only via table
- the new arguments institute and mip

Here is the list of tables for CMIP6:
https://github.com/WCRP-CMIP/CMIP6_CVs/blob/master/CMIP6_table_id.json

Basically, a table is a combination of the realm considered (A for atmosphere, O for ocean, SI for Sea Ice, etc.) and the frequency (mon for monthly, day for daily, etc.)


In [4]:
# -- Here is an example of ssp585 scenario of IPSL-CM6A-LR, monthly tas
dat_cmip6 = ds(project = 'CMIP6',
               model = 'IPSL-CM6A-LR',
               experiment = 'ssp585',
               period = '2015-2100',
               variable = 'tas',
                   # -- CMIP6 new features
                   realization = 'r1i1p1f1',
                   table= 'Amon',
                   grid = 'gr'
              )
summary(dat_cmip6)

/bdd/CMIP6/ScenarioMIP/IPSL/IPSL-CM6A-LR/ssp585/r1i1p1f1/Amon/tas/gr/latest/tas_Amon_IPSL-CM6A-LR_ssp585_r1i1p1f1_gr_201501-210012.nc


{'domain': 'global',
 'experiment': 'ssp585',
 'grid': 'gr',
 'institute': '*',
 'mip': '*',
 'model': 'IPSL-CM6A-LR',
 'period': 2015-2100,
 'project': 'CMIP6',
 'realization': 'r1i1p1f1',
 'root': '/bdd',
 'simulation': '',
 'table': 'Amon',
 'variable': 'tas',
 'version': 'latest'}

### Showing the result of automatic replacement of the '*'' by their unique values with .explore('resolve')

In [5]:
summary(dat_cmip6.explore('resolve'))

/bdd/CMIP6/ScenarioMIP/IPSL/IPSL-CM6A-LR/ssp585/r1i1p1f1/Amon/tas/gr/latest/tas_Amon_IPSL-CM6A-LR_ssp585_r1i1p1f1_gr_201501-210012.nc


{'domain': 'global',
 'experiment': 'ssp585',
 'grid': 'gr',
 'institute': 'IPSL',
 'mip': 'ScenarioMIP',
 'model': 'IPSL-CM6A-LR',
 'period': 2015-2100,
 'project': 'CMIP6',
 'realization': 'r1i1p1f1',
 'root': '/bdd',
 'simulation': '',
 'table': 'Amon',
 'variable': 'tas',
 'version': 'latest'}

## 2. Building CMIP6 CliMAF ensembles

#### Pre-requisite: the basis of CliMAF ensembles is explained in the notebook Working_with_CliMAF_ensembles.ipynb

The CMIP6 archive is slightly different from the CMIP5 archive.

The key 'institute' and the new key 'grid' implies that we can't build a CMIP6 multi-model ensemble in one move with .explore('ensemble') (example and solution below), since we easily have more than one key with multiple values.

Building a multi-realization ensemble on one model still remains an easy task.

### 2.1 Building a multi-realization ensemble on one model 

In [6]:
# -- Let's start with an easy example, with the same behavior as described
#    for the CMIP5 multi-model ensemble
req_cmip6 = ds(project='CMIP6', variable='pr', period='*',
               table='Amon', model='CNRM-CM6-1',
               realization='*'
              )

req_cmip6.explore('choices')

{'grid': 'gr',
 'institute': 'CNRM-CERFACS',
 'mip': 'CMIP',
 'period': [1850-2014],
 'realization': ['r10i1p1f2',
  'r11i1p1f2',
  'r12i1p1f2',
  'r13i1p1f2',
  'r14i1p1f2',
  'r15i1p1f2',
  'r16i1p1f2',
  'r17i1p1f2',
  'r18i1p1f2',
  'r19i1p1f2',
  'r1i1p1f2',
  'r20i1p1f2',
  'r21i1p1f2',
  'r22i1p1f2',
  'r24i1p1f2',
  'r25i1p1f2',
  'r26i1p1f2',
  'r28i1p1f2',
  'r29i1p1f2',
  'r2i1p1f2',
  'r3i1p1f2',
  'r4i1p1f2',
  'r5i1p1f2',
  'r6i1p1f2',
  'r7i1p1f2',
  'r8i1p1f2',
  'r9i1p1f2']}

In [7]:
# -- realization is the only key with a list of values, we can thus use .explore('ensemble')
#    to create an ensemble
multirealization_cmip6 = req_cmip6.explore('ensemble')

summary(multirealization_cmip6)

Keys - values:
{'domain': 'global', 'version': 'latest', 'institute': 'CNRM-CERFACS', 'realization': 'r26i1p1f2', 'period': 1850-2014, 'simulation': '', 'project': 'CMIP6', 'experiment': 'historical', 'grid': 'gr', 'table': 'Amon', 'variable': 'pr', 'mip': 'CMIP', 'model': 'CNRM-CM6-1', 'root': '/bdd'}
-- Ensemble members:
r10i1p1f2
/bdd/CMIP6/CMIP/CNRM-CERFACS/CNRM-CM6-1/historical/r10i1p1f2/Amon/pr/gr/latest/pr_Amon_CNRM-CM6-1_historical_r10i1p1f2_gr_185001-201412.nc
--
r11i1p1f2
/bdd/CMIP6/CMIP/CNRM-CERFACS/CNRM-CM6-1/historical/r11i1p1f2/Amon/pr/gr/latest/pr_Amon_CNRM-CM6-1_historical_r11i1p1f2_gr_185001-201412.nc
--
r12i1p1f2
/bdd/CMIP6/CMIP/CNRM-CERFACS/CNRM-CM6-1/historical/r12i1p1f2/Amon/pr/gr/latest/pr_Amon_CNRM-CM6-1_historical_r12i1p1f2_gr_185001-201412.nc
--
r13i1p1f2
/bdd/CMIP6/CMIP/CNRM-CERFACS/CNRM-CM6-1/historical/r13i1p1f2/Amon/pr/gr/latest/pr_Amon_CNRM-CM6-1_historical_r13i1p1f2_gr_185001-201412.nc
--
r14i1p1f2
/bdd/CMIP6/CMIP/CNRM-CERFACS/CNRM-CM6-1/historical/r14i1p

### 2.2 Building a CMIP6 multi-model ensemble
Let's now build a multi-model ensemble with only one realization (next step 2.3 is about multi-model multi-realization ensemble)

In [8]:
# -- CMIP6: multi-model ensemble on the first member r1i1p1f1
req_cmip6 = ds(project='CMIP6',
               variable='pr',
               period='2050',
               experiment='ssp585',
               table='Amon',
               model='*',
               realization='r1i1p1f1' # -- we set r1i1p1f1
              )

req_cmip6.explore('choices')

{'grid': ['gn', 'gr', 'gr1'],
 'institute': ['AWI',
  'BCC',
  'CAMS',
  'CAS',
  'CCCma',
  'CSIRO',
  'CSIRO-ARCCSS',
  'DKRZ',
  'INM',
  'IPSL',
  'MIROC',
  'MPI-M',
  'MRI',
  'NCAR',
  'NIMS-KMA',
  'NOAA-GFDL',
  'NUIST',
  'THU'],
 'mip': 'ScenarioMIP',
 'model': ['ACCESS-CM2',
  'ACCESS-ESM1-5',
  'AWI-CM-1-1-MR',
  'BCC-CSM2-MR',
  'CAMS-CSM1-0',
  'CESM2',
  'CESM2-WACCM',
  'CIESM',
  'CanESM5',
  'FGOALS-f3-L',
  'GFDL-CM4',
  'GFDL-ESM4',
  'INM-CM4-8',
  'INM-CM5-0',
  'IPSL-CM6A-LR',
  'KACE-1-0-G',
  'MIROC6',
  'MPI-ESM1-2-HR',
  'MPI-ESM1-2-LR',
  'MRI-ESM2-0',
  'NESM3']}

#### We see that we have multiple models, but also multiple institutes and grids

#### CliMAF can't associate automatically each model with one institute, we thus have to do it by hand

### We start with an 'easy' example, with the variable 'pr' for which each model has provided only one grid.
We use a try/except test on .explore('resolve') to see if CliMAF finds automatically the (potential) only grid available. If there are multiple grids available, it returns a message

In [9]:
# -- we create a 'request dictionary' with the main keyword/values (without model)
req_dict = dict(project='CMIP6',
                variable='pr',
                period='2050',
                experiment='ssp585',
                table='Amon',
                grid = 'g*',
                realization='r1i1p1f1' # -- we set r1i1p1f1
               )

# -- We make a first request to retrieve the list of models
req_cmip6 = ds(model='*', **req_dict)

list_of_models = req_cmip6.explore('choices')['model']

# -- Then, we loop on the models:
#      - we initialize an empty dictionary ens_dict
#      - for each model that has only one dataset (check with .explore('resolve')),
#        we add the dataset to ens_dict with the key 'model'
#      - after the loop, we use cens to create the ensemble from ens_dict
ens_dict = dict()
for model in list_of_models:
    try:
        ens_dict[model] = ds(model=model, **req_dict).explore('resolve')
        print model
        print ens_dict[model].baseFiles()
    except:
        print 'Check the result of .explore("choices") for model ', model
        print ds(model=model, **req_dict).explore('choices')

# -- Last step: we create the ensemble with cens
myens = cens(ens_dict)

ACCESS-CM2
/bdd/CMIP6/ScenarioMIP/CSIRO-ARCCSS/ACCESS-CM2/ssp585/r1i1p1f1/Amon/pr/gn/latest/pr_Amon_ACCESS-CM2_ssp585_r1i1p1f1_gn_201501-210012.nc
ACCESS-ESM1-5
/bdd/CMIP6/ScenarioMIP/CSIRO/ACCESS-ESM1-5/ssp585/r1i1p1f1/Amon/pr/gn/latest/pr_Amon_ACCESS-ESM1-5_ssp585_r1i1p1f1_gn_201501-210012.nc
AWI-CM-1-1-MR
/bdd/CMIP6/ScenarioMIP/AWI/AWI-CM-1-1-MR/ssp585/r1i1p1f1/Amon/pr/gn/latest/pr_Amon_AWI-CM-1-1-MR_ssp585_r1i1p1f1_gn_205001-205012.nc
BCC-CSM2-MR
/bdd/CMIP6/ScenarioMIP/BCC/BCC-CSM2-MR/ssp585/r1i1p1f1/Amon/pr/gn/latest/pr_Amon_BCC-CSM2-MR_ssp585_r1i1p1f1_gn_201501-210012.nc
CAMS-CSM1-0
/bdd/CMIP6/ScenarioMIP/CAMS/CAMS-CSM1-0/ssp585/r1i1p1f1/Amon/pr/gn/latest/pr_Amon_CAMS-CSM1-0_ssp585_r1i1p1f1_gn_201501-209912.nc
CESM2
/bdd/CMIP6/ScenarioMIP/NCAR/CESM2/ssp585/r1i1p1f1/Amon/pr/gn/latest/pr_Amon_CESM2_ssp585_r1i1p1f1_gn_201501-206412.nc
CESM2-WACCM
/bdd/CMIP6/ScenarioMIP/NCAR/CESM2-WACCM/ssp585/r1i1p1f1/Amon/pr/gn/latest/pr_Amon_CESM2-WACCM_ssp585_r1i1p1f1_gn_201501-210012.nc
CIESM
/b

### Now we use the variable 'tos' for which many models have provided multiple grids; we can see that we need to choose among the available grids (next step below)

In [10]:
# -- we create a 'request dictionary' with the main keyword/values (without model)
req_dict = dict(project='CMIP6',
                variable='tos',
                period='2050',
                experiment='ssp585',
                table='Omon',
                grid = 'g*',
                realization='r1i1p1f1' # -- we set r1i1p1f1
               )

# -- We make a first request to retrieve the list of models
req_cmip6 = ds(model='*', **req_dict)

list_of_models = req_cmip6.explore('choices')['model']

# -- Then, we loop on the models 
ens_dict = dict()
for model in list_of_models:
    try:
        ens_dict[model] = ds(model=model, **req_dict).explore('resolve')
        print model
        print ens_dict[model].baseFiles()
    except:
        print 'Check the result of .explore("choices") for model ', model
        print ds(model=model, **req_dict).explore('choices')

# -- Last step: we create the ensemble with cens
myens = cens(ens_dict)

BCC-CSM2-MR
/bdd/CMIP6/ScenarioMIP/BCC/BCC-CSM2-MR/ssp585/r1i1p1f1/Omon/tos/gn/latest/tos_Omon_BCC-CSM2-MR_ssp585_r1i1p1f1_gn_201501-210012.nc
CAMS-CSM1-0
/bdd/CMIP6/ScenarioMIP/CAMS/CAMS-CSM1-0/ssp585/r1i1p1f1/Omon/tos/gn/latest/tos_Omon_CAMS-CSM1-0_ssp585_r1i1p1f1_gn_201501-209912.nc
Check the result of .explore("choices") for model  CESM2
{'institute': 'NCAR', 'mip': 'ScenarioMIP', 'grid': ['gn', 'gr']}
Check the result of .explore("choices") for model  CESM2-WACCM
{'institute': 'NCAR', 'mip': 'ScenarioMIP', 'grid': ['gn', 'gr']}
CIESM
/bdd/CMIP6/ScenarioMIP/THU/CIESM/ssp585/r1i1p1f1/Omon/tos/gn/latest/tos_Omon_CIESM_ssp585_r1i1p1f1_gn_201501-206412.nc
CanESM5
/bdd/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp585/r1i1p1f1/Omon/tos/gn/latest/tos_Omon_CanESM5_ssp585_r1i1p1f1_gn_201501-210012.nc
FGOALS-f3-L
/bdd/CMIP6/ScenarioMIP/CAS/FGOALS-f3-L/ssp585/r1i1p1f1/Omon/tos/gn/latest/tos_Omon_FGOALS-f3-L_ssp585_r1i1p1f1_gn_201501-206412.nc
FIO-ESM-2-0
/bdd/CMIP6/ScenarioMIP/FIO-QLNM/FIO-ESM-2-0/ssp5

### We now implement a way to choose the grid automatically from a list of grids 'mygrids' (in order of priority)

In [11]:
# -- we create a 'request dictionary' with the main keyword/values (without model)
req_dict = dict(project='CMIP6',
                variable='tos',
                period='2050',
                experiment='ssp585',
                table='Omon',
                grid = 'g*',
                realization='r1i1p1f1' # -- we set r1i1p1f1
               )

# -- We make a first request to retrieve the list of models
req_cmip6 = ds(model='*', **req_dict)

# -- Retrieve the list of models
list_of_models = req_cmip6.explore('choices')['model']

# -- We make a list of grids in order of preference
mygrids = ['gn','gr','gr1','gr2']

# -- Then, we loop on the models 
ens_dict = dict()
for model in list_of_models:
    # -- If we find a matching dataset, we add it to ens_dict
    try:
        ens_dict[model] = ds(model=model, **req_dict).explore('resolve')
        print model
    except:
        # -- if there are multiple grids available:
        #     - we retrieve the list of available grids
        #     - and keep the first grid from mygrids that is available 
        available_grids = ds(model=model, **req_dict).explore('choices')['grid']
        if isinstance(available_grids,list):
            # -- Loop on my own list of grids and stop at the first available
            for mygrid in mygrids:
                if mygrid in available_grids:
                    break
            tmp_req_dict = req_dict.copy()
            tmp_req_dict['grid'] = mygrid
            ens_dict[model] = ds(model=model, **tmp_req_dict).explore('resolve')
        print 'Picked grid = ',mygrid, 'for model', model
    print ens_dict[model].baseFiles()

# -- Create ensemble
myens = cens(ens_dict)

BCC-CSM2-MR
/bdd/CMIP6/ScenarioMIP/BCC/BCC-CSM2-MR/ssp585/r1i1p1f1/Omon/tos/gn/latest/tos_Omon_BCC-CSM2-MR_ssp585_r1i1p1f1_gn_201501-210012.nc
CAMS-CSM1-0
/bdd/CMIP6/ScenarioMIP/CAMS/CAMS-CSM1-0/ssp585/r1i1p1f1/Omon/tos/gn/latest/tos_Omon_CAMS-CSM1-0_ssp585_r1i1p1f1_gn_201501-209912.nc
Picked grid =  gn for model CESM2
/bdd/CMIP6/ScenarioMIP/NCAR/CESM2/ssp585/r1i1p1f1/Omon/tos/gn/latest/tos_Omon_CESM2_ssp585_r1i1p1f1_gn_201501-206412.nc
Picked grid =  gn for model CESM2-WACCM
/bdd/CMIP6/ScenarioMIP/NCAR/CESM2-WACCM/ssp585/r1i1p1f1/Omon/tos/gn/latest/tos_Omon_CESM2-WACCM_ssp585_r1i1p1f1_gn_201501-210012.nc
CIESM
/bdd/CMIP6/ScenarioMIP/THU/CIESM/ssp585/r1i1p1f1/Omon/tos/gn/latest/tos_Omon_CIESM_ssp585_r1i1p1f1_gn_201501-206412.nc
CanESM5
/bdd/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp585/r1i1p1f1/Omon/tos/gn/latest/tos_Omon_CanESM5_ssp585_r1i1p1f1_gn_201501-210012.nc
FGOALS-f3-L
/bdd/CMIP6/ScenarioMIP/CAS/FGOALS-f3-L/ssp585/r1i1p1f1/Omon/tos/gn/latest/tos_Omon_FGOALS-f3-L_ssp585_r1i1p1f1_gn_201

### 2.3 Select the first member available for each model, without specifying one particular realization for all the models

In [12]:
# -- we create a 'request dictionary' with the main keyword/values (without model)
req_dict = dict(project='CMIP6',
                variable='tos',
                period='2050',
                experiment='ssp585',
                table='Omon',
                grid = 'g*',
                realization='*' # -- we set r1i1p1f1
               )

# -- We make a first request to retrieve the list of models
req_cmip6 = ds(model='*', **req_dict)

list_of_models = req_cmip6.explore('choices')['model']

# -- We make a list of grids in order of preference
mygrids = ['gn','gr','gr1','gr2']
ens_dict = dict()

from natsort import natsorted

# -- Then, we loop on the models 
for model in list_of_models:
    tmp_req_dict = req_dict.copy()
    
    available_choices_for_model = ds(model=model, **req_dict).explore('choices')
    available_grids = available_choices_for_model['grid']
    if isinstance(available_grids,list):
        # -- Loop on my own list of grids and stop at the first available
        for mygrid in mygrids:
            if mygrid in available_grids:
                break
    else:
        mygrid = available_grids
    #
    # -- We retrieve the available realizations:
    #     - if it's a list, we take the first one from a 'natsorted' list
    #     - if not, we take the available dataset
    available_realizations = available_choices_for_model['realization']
    if isinstance(available_realizations, list):
        tmp_req_dict['realization'] = natsorted(available_realizations)[0]
    else:
        tmp_req_dict['realization'] = available_realizations

    
    #tmp_req_dict['realization'] = available_realizations[0]
    tmp_req_dict['grid'] = mygrid
    
    ens_dict[model] = ds(model=model, **tmp_req_dict).explore('resolve')
    print model,';', ens_dict[model].baseFiles()


BCC-CSM2-MR ; /bdd/CMIP6/ScenarioMIP/BCC/BCC-CSM2-MR/ssp585/r1i1p1f1/Omon/tos/gn/latest/tos_Omon_BCC-CSM2-MR_ssp585_r1i1p1f1_gn_201501-210012.nc
CAMS-CSM1-0 ; /bdd/CMIP6/ScenarioMIP/CAMS/CAMS-CSM1-0/ssp585/r1i1p1f1/Omon/tos/gn/latest/tos_Omon_CAMS-CSM1-0_ssp585_r1i1p1f1_gn_201501-209912.nc
CESM2 ; /bdd/CMIP6/ScenarioMIP/NCAR/CESM2/ssp585/r1i1p1f1/Omon/tos/gn/latest/tos_Omon_CESM2_ssp585_r1i1p1f1_gn_201501-206412.nc
CESM2-WACCM ; /bdd/CMIP6/ScenarioMIP/NCAR/CESM2-WACCM/ssp585/r1i1p1f1/Omon/tos/gn/latest/tos_Omon_CESM2-WACCM_ssp585_r1i1p1f1_gn_201501-210012.nc
CIESM ; /bdd/CMIP6/ScenarioMIP/THU/CIESM/ssp585/r1i1p1f1/Omon/tos/gn/latest/tos_Omon_CIESM_ssp585_r1i1p1f1_gn_201501-206412.nc
CNRM-CM6-1 ; /bdd/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM-CM6-1/ssp585/r1i1p1f2/Omon/tos/gn/latest/tos_Omon_CNRM-CM6-1_ssp585_r1i1p1f2_gn_201501-210012.nc
CNRM-CM6-1-HR ; /bdd/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM-CM6-1-HR/ssp585/r1i1p1f2/Omon/tos/gn/latest/tos_Omon_CNRM-CM6-1-HR_ssp585_r1i1p1f2_gn_201501-210012.

### 2.4 Meta-ensemble with all models and all realizations available

In [13]:
# -- we create a 'request dictionary' with the main keyword/values (without model)
req_dict = dict(project='CMIP6',
                variable='tos',
                period='2050',
                experiment='ssp585',
                table='Omon',
                grid = 'g*',
                realization='*' # -- we set r1i1p1f1
               )

# -- We make a first request to retrieve the list of models
req_cmip6 = ds(model='*', **req_dict)

list_of_models = req_cmip6.explore('choices')['model']

# -- We make a list of grids in order of preference
mygrids = ['gn','gr','gr1','gr2']
ens_dict = dict()

from natsort import natsorted

# -- Then, we loop on the models 
for model in list_of_models:
    available_choices_for_model = ds(model=model, **req_dict).explore('choices')
    available_grids = available_choices_for_model['grid']
    if isinstance(available_grids,list):
        # -- Loop on my own list of grids and stop at the first available
        for mygrid in mygrids:
            if mygrid in available_grids:
                break
    else:
        mygrid = available_grids
    
    if isinstance(available_choices_for_model['realization'], list):
        available_realizations = natsorted(available_choices_for_model['realization'])
    else:
        available_realizations = [available_choices_for_model['realization']]
    
    print model,'==>'
    for realization in available_realizations:
        tmp_req_dict = req_dict.copy()
        tmp_req_dict['model'] = model
        tmp_req_dict['realization'] = realization
        tmp_req_dict['grid'] = mygrid
    
        ens_dict[model+'_'+realization] = ds(**tmp_req_dict).explore('resolve')
        print realization, ' = ', ens_dict[model+'_'+realization].baseFiles()
#
# -- and we create the meta-ensemble
meta_ensemble = cens(ens_dict)

BCC-CSM2-MR ==>
r1i1p1f1  =  /bdd/CMIP6/ScenarioMIP/BCC/BCC-CSM2-MR/ssp585/r1i1p1f1/Omon/tos/gn/latest/tos_Omon_BCC-CSM2-MR_ssp585_r1i1p1f1_gn_201501-210012.nc
CAMS-CSM1-0 ==>
r1i1p1f1  =  /bdd/CMIP6/ScenarioMIP/CAMS/CAMS-CSM1-0/ssp585/r1i1p1f1/Omon/tos/gn/latest/tos_Omon_CAMS-CSM1-0_ssp585_r1i1p1f1_gn_201501-209912.nc
r2i1p1f1  =  /bdd/CMIP6/ScenarioMIP/CAMS/CAMS-CSM1-0/ssp585/r2i1p1f1/Omon/tos/gn/latest/tos_Omon_CAMS-CSM1-0_ssp585_r2i1p1f1_gn_201501-209912.nc
CESM2 ==>
r1i1p1f1  =  /bdd/CMIP6/ScenarioMIP/NCAR/CESM2/ssp585/r1i1p1f1/Omon/tos/gn/latest/tos_Omon_CESM2_ssp585_r1i1p1f1_gn_201501-206412.nc
r2i1p1f1  =  /bdd/CMIP6/ScenarioMIP/NCAR/CESM2/ssp585/r2i1p1f1/Omon/tos/gn/latest/tos_Omon_CESM2_ssp585_r2i1p1f1_gn_201501-206412.nc
r10i1p1f1  =  /bdd/CMIP6/ScenarioMIP/NCAR/CESM2/ssp585/r10i1p1f1/Omon/tos/gn/latest/tos_Omon_CESM2_ssp585_r10i1p1f1_gn_201501-206412.nc
r11i1p1f1  =  /bdd/CMIP6/ScenarioMIP/NCAR/CESM2/ssp585/r11i1p1f1/Omon/tos/gn/latest/tos_Omon_CESM2_ssp585_r11i1p1f1_gn_201

r19i1p1f1  =  /bdd/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp585/r19i1p1f1/Omon/tos/gn/latest/tos_Omon_CanESM5_ssp585_r19i1p1f1_gn_201501-210012.nc
r19i1p2f1  =  /bdd/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp585/r19i1p2f1/Omon/tos/gn/latest/tos_Omon_CanESM5_ssp585_r19i1p2f1_gn_201501-210012.nc
r20i1p1f1  =  /bdd/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp585/r20i1p1f1/Omon/tos/gn/latest/tos_Omon_CanESM5_ssp585_r20i1p1f1_gn_201501-210012.nc
r20i1p2f1  =  /bdd/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp585/r20i1p2f1/Omon/tos/gn/latest/tos_Omon_CanESM5_ssp585_r20i1p2f1_gn_201501-210012.nc
r21i1p1f1  =  /bdd/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp585/r21i1p1f1/Omon/tos/gn/latest/tos_Omon_CanESM5_ssp585_r21i1p1f1_gn_201501-210012.nc
r21i1p2f1  =  /bdd/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp585/r21i1p2f1/Omon/tos/gn/latest/tos_Omon_CanESM5_ssp585_r21i1p2f1_gn_201501-210012.nc
r22i1p1f1  =  /bdd/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp585/r22i1p1f1/Omon/tos/gn/latest/tos_Omon_CanESM5_ssp585_r22i1p1f1_gn_201501-210012.nc
r22i1p2f1  = 

### Tip: if you want to separate the model and split this ensemble in multi-realization ensembles (one for each model), a solution is to make a dictionnary of ensembles:

In [14]:
# -- we create a 'request dictionary' with the main keyword/values (without model)
req_dict = dict(project='CMIP6',
                variable='tos',
                period='2050',
                experiment='ssp585',
                table='Omon',
                grid = 'g*',
                realization='*' # -- we set r1i1p1f1
               )

# -- We make a first request to retrieve the list of models
req_cmip6 = ds(model='*', **req_dict)

list_of_models = req_cmip6.explore('choices')['model']

# -- We make a list of grids in order of preference
mygrids = ['gn','gr','gr1','gr2']
model_ens_dict = dict()

from natsort import natsorted

# -- Then, we loop on the models 
for model in list_of_models:
    
    ens_dict = dict()
    
    available_choices_for_model = ds(model=model, **req_dict).explore('choices')
    available_grids = available_choices_for_model['grid']
    if isinstance(available_grids,list):
        # -- Loop on my own list of grids and stop at the first available
        for mygrid in mygrids:
            if mygrid in available_grids:
                break
    else:
        mygrid = available_grids
    
    if isinstance(available_choices_for_model['realization'], list):
        available_realizations = natsorted(available_choices_for_model['realization'])
    else:
        available_realizations = [available_choices_for_model['realization']]
    
    print model,'==>'
    for realization in available_realizations:
        tmp_req_dict = req_dict.copy()
        tmp_req_dict['model'] = model
        tmp_req_dict['realization'] = realization
        tmp_req_dict['grid'] = mygrid
    
        ens_dict[realization] = ds(**tmp_req_dict).explore('resolve')
        print realization
    #
    # -- CliMAF doesn't work on 'one-member ensembles';
    #    if there is only one member for the model,
    #    don't make an ensemble with cens and simply provide the dictionary ens_dict
    #    !! it implies that you will have to separate both cases
    #    ('one-member' model or 'multi-realization' model) when you will do your analyses
    if len(available_realizations)==1:
        model_ens_dict[model] = ens_dict
    else:
        model_ens_dict[model] = cens(ens_dict, order=available_realizations)

BCC-CSM2-MR ==>
r1i1p1f1
CAMS-CSM1-0 ==>
r1i1p1f1
r2i1p1f1
CESM2 ==>
r1i1p1f1
r2i1p1f1
r10i1p1f1
r11i1p1f1
CESM2-WACCM ==>
r1i1p1f1
r2i1p1f1
r5i1p1f1
CIESM ==>
r1i1p1f1
CNRM-CM6-1 ==>
r1i1p1f2
r2i1p1f2
r3i1p1f2
r4i1p1f2
r5i1p1f2
r6i1p1f2
CNRM-CM6-1-HR ==>
r1i1p1f2
CNRM-ESM2-1 ==>
r1i1p1f2
r2i1p1f2
r3i1p1f2
r4i1p1f2
r5i1p1f2
CanESM5 ==>
r1i1p1f1
r1i1p2f1
r2i1p1f1
r2i1p2f1
r3i1p1f1
r3i1p2f1
r4i1p1f1
r4i1p2f1
r5i1p1f1
r5i1p2f1
r6i1p1f1
r6i1p2f1
r7i1p1f1
r7i1p2f1
r8i1p1f1
r8i1p2f1
r9i1p1f1
r9i1p2f1
r10i1p1f1
r10i1p2f1
r11i1p1f1
r11i1p2f1
r12i1p1f1
r12i1p2f1
r13i1p1f1
r13i1p2f1
r14i1p1f1
r14i1p2f1
r15i1p1f1
r15i1p2f1
r16i1p1f1
r16i1p2f1
r17i1p1f1
r17i1p2f1
r18i1p1f1
r18i1p2f1
r19i1p1f1
r19i1p2f1
r20i1p1f1
r20i1p2f1
r21i1p1f1
r21i1p2f1
r22i1p1f1
r22i1p2f1
r23i1p1f1
r23i1p2f1
r24i1p1f1
r24i1p2f1
r25i1p1f1
r25i1p2f1
CanESM5-CanOE ==>
r1i1p2f1
r2i1p2f1
r3i1p2f1
FGOALS-f3-L ==>
r1i1p1f1
r2i1p1f1
FGOALS-g3 ==>
r2i1p1f1
r4i1p1f1
FIO-ESM-2-0 ==>
r1i1p1f1
GFDL-CM4 ==>
r1i1p1f1
GISS-E2-1-G ==>
r1i1p1

### Here is the result: a dictionary with elements that are ensembles

In [15]:
model_ens_dict

{'BCC-CSM2-MR': {'r1i1p1f1': ds('CMIP6%%tos%2050%global%/bdd%BCC-CSM2-MR%BCC%ScenarioMIP%Omon%ssp585%r1i1p1f1%gn%latest')},
 'CAMS-CSM1-0': cens({'r1i1p1f1':ds('CMIP6%%tos%2050%global%/bdd%CAMS-CSM1-0%CAMS%ScenarioMIP%Omon%ssp585%r1i1p1f1%gn%latest'),'r2i1p1f1':ds('CMIP6%%tos%2050%global%/bdd%CAMS-CSM1-0%CAMS%ScenarioMIP%Omon%ssp585%r2i1p1f1%gn%latest')}),
 'CESM2': cens({'r1i1p1f1':ds('CMIP6%%tos%2050%global%/bdd%CESM2%NCAR%ScenarioMIP%Omon%ssp585%r1i1p1f1%gn%latest'),'r2i1p1f1':ds('CMIP6%%tos%2050%global%/bdd%CESM2%NCAR%ScenarioMIP%Omon%ssp585%r2i1p1f1%gn%latest'),'r10i1p1f1':ds('CMIP6%%tos%2050%global%/bdd%CESM2%NCAR%ScenarioMIP%Omon%ssp585%r10i1p1f1%gn%latest'),'r11i1p1f1':ds('CMIP6%%tos%2050%global%/bdd%CESM2%NCAR%ScenarioMIP%Omon%ssp585%r11i1p1f1%gn%latest')}),
 'CESM2-WACCM': cens({'r1i1p1f1':ds('CMIP6%%tos%2050%global%/bdd%CESM2-WACCM%NCAR%ScenarioMIP%Omon%ssp585%r1i1p1f1%gn%latest'),'r2i1p1f1':ds('CMIP6%%tos%2050%global%/bdd%CESM2-WACCM%*%*%Omon%ssp585%r2i1p1f1%gn%latest'),'r5

## 3. The IPSLCM6_historical-EXT project

### The IPSL ran a set of extensions of the historical run members until 2060 with the ssp585 scenario. This ensemble is available via the project IPSL-CM6_historical-EXT

In [16]:
dat_histEXT = ds(project='IPSL-CM6_historical-EXT',
                 variable='tas',
                 table='Amon',
                 period='1950-2060',
                 realization='r1i1p1f1'
                )
summary(dat_histEXT)

/bdd/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical-EXT/r1i1p1f1/Amon/tas/gr/latest/tas_Amon_IPSL-CM6A-LR_historical_r1i1p1f1_gr_185001-201412.nc
/bdd/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical-EXT/r1i1p1f1/Amon/tas/gr/latest/tas_Amon_IPSL-CM6A-LR_historical_r1i1p1f1_gr_201501-202912.nc
/bdd/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical-EXT/r1i1p1f1/Amon/tas/gr/latest/tas_Amon_IPSL-CM6A-LR_historical_r1i1p1f1_gr_203001-205912.nc


{'domain': 'global',
 'experiment': 'historical',
 'grid': 'g*',
 'institute': 'IPSL',
 'mip': 'CMIP',
 'model': 'IPSL-CM6A-LR',
 'period': 1950-2060,
 'project': 'IPSL-CM6_historical-EXT',
 'realization': 'r1i1p1f1',
 'root': '/bdd',
 'simulation': '',
 'table': 'Amon',
 'variable': 'tas',
 'version': 'latest'}

In [17]:
from climaf.api import *

### Let's now work with the full Large Ensemble:

In [18]:
req_histEXT = ds(project='IPSL-CM6_historical-EXT',
          variable='tas',
          table='Amon',
          period='1950-2060',
          realization='*'
         )
req_histEXT.explore('choices')

{'grid': 'gr',
 'realization': ['r10i1p1f1',
  'r11i1p1f1',
  'r12i1p1f1',
  'r13i1p1f1',
  'r14i1p1f1',
  'r15i1p1f1',
  'r16i1p1f1',
  'r17i1p1f1',
  'r18i1p1f1',
  'r19i1p1f1',
  'r1i1p1f1',
  'r20i1p1f1',
  'r21i1p1f1',
  'r22i1p1f1',
  'r23i1p1f1',
  'r24i1p1f1',
  'r25i1p1f1',
  'r26i1p1f1',
  'r27i1p1f1',
  'r28i1p1f1',
  'r29i1p1f1',
  'r2i1p1f1',
  'r30i1p1f1',
  'r31i1p1f1',
  'r32i1p1f1',
  'r3i1p1f1',
  'r4i1p1f1',
  'r5i1p1f1',
  'r6i1p1f1',
  'r7i1p1f1',
  'r8i1p1f1',
  'r9i1p1f1']}

In [19]:
ens_histEXT = req_histEXT.explore('ensemble')
summary(ens_histEXT)

Keys - values:
{'domain': 'global', 'version': 'latest', 'institute': 'IPSL', 'realization': 'r10i1p1f1', 'period': 1950-2060, 'simulation': '', 'project': 'IPSL-CM6_historical-EXT', 'experiment': 'historical', 'grid': 'gr', 'table': 'Amon', 'variable': 'tas', 'mip': 'CMIP', 'model': 'IPSL-CM6A-LR', 'root': '/bdd'}
-- Ensemble members:
r10i1p1f1
/bdd/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical-EXT/r10i1p1f1/Amon/tas/gr/latest/tas_Amon_IPSL-CM6A-LR_historical_r10i1p1f1_gr_185001-201412.nc
/bdd/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical-EXT/r10i1p1f1/Amon/tas/gr/latest/tas_Amon_IPSL-CM6A-LR_historical_r10i1p1f1_gr_201501-202912.nc
/bdd/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical-EXT/r10i1p1f1/Amon/tas/gr/latest/tas_Amon_IPSL-CM6A-LR_historical_r10i1p1f1_gr_203001-205912.nc
--
r11i1p1f1
/bdd/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical-EXT/r11i1p1f1/Amon/tas/gr/latest/tas_Amon_IPSL-CM6A-LR_historical_r11i1p1f1_gr_185001-201412.nc
/bdd/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical-EXT/r11i1p1f1/Amon/tas/gr/late

/bdd/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical-EXT/r30i1p1f1/Amon/tas/gr/latest/tas_Amon_IPSL-CM6A-LR_historical_r30i1p1f1_gr_185001-201412.nc
/bdd/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical-EXT/r30i1p1f1/Amon/tas/gr/latest/tas_Amon_IPSL-CM6A-LR_historical_r30i1p1f1_gr_201501-202912.nc
/bdd/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical-EXT/r30i1p1f1/Amon/tas/gr/latest/tas_Amon_IPSL-CM6A-LR_historical_r30i1p1f1_gr_203001-205912.nc
--
r31i1p1f1
/bdd/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical-EXT/r31i1p1f1/Amon/tas/gr/latest/tas_Amon_IPSL-CM6A-LR_historical_r31i1p1f1_gr_185001-201412.nc
/bdd/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical-EXT/r31i1p1f1/Amon/tas/gr/latest/tas_Amon_IPSL-CM6A-LR_historical_r31i1p1f1_gr_201501-202912.nc
/bdd/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical-EXT/r31i1p1f1/Amon/tas/gr/latest/tas_Amon_IPSL-CM6A-LR_historical_r31i1p1f1_gr_203001-205912.nc
--
r32i1p1f1
/bdd/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical-EXT/r32i1p1f1/Amon/tas/gr/latest/tas_Amon_IPSL-CM6A-LR_historical_r32i1p1f1_gr_1850

### Tip: ordering the members with natsort

In [20]:
from natsort import natsorted
ens_histEXT.order = natsorted(ens_histEXT.order)
summary(ens_histEXT)

Keys - values:
{'domain': 'global', 'version': 'latest', 'institute': 'IPSL', 'realization': 'r10i1p1f1', 'period': 1950-2060, 'simulation': '', 'project': 'IPSL-CM6_historical-EXT', 'experiment': 'historical', 'grid': 'gr', 'table': 'Amon', 'variable': 'tas', 'mip': 'CMIP', 'model': 'IPSL-CM6A-LR', 'root': '/bdd'}
-- Ensemble members:
r1i1p1f1
/bdd/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical-EXT/r1i1p1f1/Amon/tas/gr/latest/tas_Amon_IPSL-CM6A-LR_historical_r1i1p1f1_gr_185001-201412.nc
/bdd/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical-EXT/r1i1p1f1/Amon/tas/gr/latest/tas_Amon_IPSL-CM6A-LR_historical_r1i1p1f1_gr_201501-202912.nc
/bdd/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical-EXT/r1i1p1f1/Amon/tas/gr/latest/tas_Amon_IPSL-CM6A-LR_historical_r1i1p1f1_gr_203001-205912.nc
--
r2i1p1f1
/bdd/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical-EXT/r2i1p1f1/Amon/tas/gr/latest/tas_Amon_IPSL-CM6A-LR_historical_r2i1p1f1_gr_185001-201412.nc
/bdd/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical-EXT/r2i1p1f1/Amon/tas/gr/latest/tas_Amon

#   
## This was how to work with CMIP6 ensembles with CliMAF!
##  
##  